In [85]:
import pandas as pd
import math
import string
import re
import nltk
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\denze\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [2]:
imdb = pd.read_csv('IMDB_Full_Reviews.csv', index_col = 0)

In [3]:
imdb.shape

(123549, 2)

In [4]:
imdb.head()

,Review,Rating
0,I'll admit I raised an eyebrow when I saw that...,10.0
1,". . . The Riddle Maker, whose real name gets u...",9.0
2,"The Riddler(Paul Dano, spot-on. How did it tak...",9.0
3,Batman helps Lieutenant James Gordon investiga...,7.0
4,"THE BATMAN (2022) *** Robert Pattinson, Zoe Kr...",8.0


In [5]:
imdb['Rating'].value_counts()

10.0    41109
9.0     16023
8.0     13342
1.0     11111
7.0      9782
6.0      7678
5.0      6458
3.0      5114
2.0      4909
4.0      4861
Name: Rating, dtype: int64

In [6]:
imdb.tail()

,Review,Rating
123544,Loved it! I love Happy Madison films. Adam San...,9.0
123545,It could have truly been a great movie about t...,1.0
123546,"This film made me depressed, I will never get ...",1.0
123547,If it wasn't about football I'd give it a 2! H...,3.0
123548,They all played a different personality than w...,10.0


In [7]:
imdb.isnull().sum()

Review       0
Rating    3162
dtype: int64

In [8]:
imdb.dropna(inplace = True)
imdb.isnull().sum()

Review    0
Rating    0
dtype: int64

In [9]:
imdb['Rating'].value_counts()

10.0    41109
9.0     16023
8.0     13342
1.0     11111
7.0      9782
6.0      7678
5.0      6458
3.0      5114
2.0      4909
4.0      4861
Name: Rating, dtype: int64

In [10]:
imdb.drop(imdb[(imdb['Rating'] > 3) & (imdb['Rating'] < 8)].index, inplace =True)

In [11]:
imdb['Rating'].value_counts()

10.0    41109
9.0     16023
8.0     13342
1.0     11111
3.0      5114
2.0      4909
Name: Rating, dtype: int64

In [12]:
def map_rating(rating):

    if type(rating) == str:
        if rating == 'negative':
            return 0 # Negative sentiment
        else:
            return 1 # Positive sentiment
    else:
        if rating <= 3:
            return 0 
        else:
            return 1 
        
imdb['Rating'] = imdb['Rating'].apply(map_rating)
imdb.rename(columns = {'Review':'review', 'Rating':'sentiment'}, inplace = True)

In [13]:
imdb.head()

,review,sentiment
0,I'll admit I raised an eyebrow when I saw that...,1
1,". . . The Riddle Maker, whose real name gets u...",1
2,"The Riddler(Paul Dano, spot-on. How did it tak...",1
4,"THE BATMAN (2022) *** Robert Pattinson, Zoe Kr...",1
6,"Always been a ""Batman"" fan as the D. C. legend...",1


In [14]:
imdb['sentiment'].value_counts()

1    70474
0    21134
Name: sentiment, dtype: int64

In [15]:
imdb_50k = pd.read_csv('IMDB Dataset.csv')

In [16]:
imdb_50k['sentiment'].value_counts()

negative    25000
positive    25000
Name: sentiment, dtype: int64

In [17]:
imdb_50k.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [18]:
imdb_50k['sentiment'] = imdb_50k['sentiment'].apply(map_rating)
imdb_50k['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [19]:
imdb_50k.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [20]:
imdb_neg = imdb_50k[imdb_50k['sentiment'] == 0]

In [21]:
imdb_neg['sentiment'].value_counts()

0    25000
Name: sentiment, dtype: int64

In [22]:
imdb = pd.concat([imdb, imdb_neg]).reset_index(drop = True)

In [23]:
imdb.shape

(116608, 2)

In [24]:
imdb.head()

,review,sentiment
0,I'll admit I raised an eyebrow when I saw that...,1
1,". . . The Riddle Maker, whose real name gets u...",1
2,"The Riddler(Paul Dano, spot-on. How did it tak...",1
3,"THE BATMAN (2022) *** Robert Pattinson, Zoe Kr...",1
4,"Always been a ""Batman"" fan as the D. C. legend...",1


In [25]:
imdb['sentiment'].value_counts()

1    70474
0    46134
Name: sentiment, dtype: int64

In [86]:
stpwrds = set(stopwords.words('english'))

In [87]:
def remove_punc(text):
    # Replace sentence-ending punctuation with whitespace
    text = text.replace('.', ' ')
    text = text.replace('!', ' ')
    text = text.replace('?', ' ')
    # Replace commas with whitespace
    text = text.replace(',', ' ')
    # Replace word-dividing punctuation with whitespace to separate words
    text = text.replace('/', ' ')
    text = text.replace('-', ' ')
    # Remove the remaining punctuation from the reviews
    text = text.translate(str.maketrans('','', string.punctuation))
    return text

def fix_space(text):
    # Fix double spaces caused by replacing punctuation, and any other weird spaces
    text = text.split()
    text = " ".join(text)
    return text

def remove_html(text):
    # Remove line breaks
    text = text.replace('<br />', '')
    pattern = re.compile('<.*?>')
    text = pattern.sub(r'', text)
    return text

def remove_stopwords(text):
    text = " ".join([string for string in str(text).split() if string not in stpwrds])
    return text
    
def clean_text(text):
    text = text.lower()
    return text

In [88]:
test = imdb_50k['review'][43]
print(test)

I had heard good things about "States of Grace" and came in with an open mind. I thought that "God's Army" was okay, and I thought that maybe Dutcher had improved and matured as a filmmaker. The film began with some shaky acting, and I thought, "well, maybe it will get better." Unfortunately, it never did. The picture starts out by introducing two elders -- Mormon missionaries -- and it seems that the audience will get to know them and grow to care about them. Instead, the story degenerates into a highly improbable series of unfortunate events highlighting blatant disobedience by the missionaries (something that undeniably exists, but rarely on the level that Dutcher portrays) and it becomes almost laughable.<br /><br />Dutcher's only success in this movie is his successful alienation of his target audience. By unrealistically and inaccurately portraying the lives of Mormon missionaries, Dutcher accomplishes nothing more than angering his predominantly Mormon audience. The film in no w

In [89]:
x = remove_html(test)
print(x)

I had heard good things about "States of Grace" and came in with an open mind. I thought that "God's Army" was okay, and I thought that maybe Dutcher had improved and matured as a filmmaker. The film began with some shaky acting, and I thought, "well, maybe it will get better." Unfortunately, it never did. The picture starts out by introducing two elders -- Mormon missionaries -- and it seems that the audience will get to know them and grow to care about them. Instead, the story degenerates into a highly improbable series of unfortunate events highlighting blatant disobedience by the missionaries (something that undeniably exists, but rarely on the level that Dutcher portrays) and it becomes almost laughable.Dutcher's only success in this movie is his successful alienation of his target audience. By unrealistically and inaccurately portraying the lives of Mormon missionaries, Dutcher accomplishes nothing more than angering his predominantly Mormon audience. The film in no way reflects 

In [90]:
x = remove_stopwords(x)
print(x)

I heard good things "States Grace" came open mind. I thought "God's Army" okay, I thought maybe Dutcher improved matured filmmaker. The film began shaky acting, I thought, "well, maybe get better." Unfortunately, never did. The picture starts introducing two elders -- Mormon missionaries -- seems audience get know grow care them. Instead, story degenerates highly improbable series unfortunate events highlighting blatant disobedience missionaries (something undeniably exists, rarely level Dutcher portrays) becomes almost laughable.Dutcher's success movie successful alienation target audience. By unrealistically inaccurately portraying lives Mormon missionaries, Dutcher accomplishes nothing angering predominantly Mormon audience. The film way reflects reality. Missions nothing like Dutcher shows (having served Mormon mission I attest fact) gang life California certainly contains much explicit language occasional mild vulgarity.The conclusion, I'm assuming supposed touch audience inspire 

In [91]:
x = remove_punc(x)
print(x)

I heard good things States Grace came open mind  I thought Gods Army okay  I thought maybe Dutcher improved matured filmmaker  The film began shaky acting  I thought  well  maybe get better  Unfortunately  never did  The picture starts introducing two elders    Mormon missionaries    seems audience get know grow care them  Instead  story degenerates highly improbable series unfortunate events highlighting blatant disobedience missionaries something undeniably exists  rarely level Dutcher portrays becomes almost laughable Dutchers success movie successful alienation target audience  By unrealistically inaccurately portraying lives Mormon missionaries  Dutcher accomplishes nothing angering predominantly Mormon audience  The film way reflects reality  Missions nothing like Dutcher shows having served Mormon mission I attest fact gang life California certainly contains much explicit language occasional mild vulgarity The conclusion  Im assuming supposed touch audience inspire believe forgi

In [92]:
print(fix_space(x))

I heard good things States Grace came open mind I thought Gods Army okay I thought maybe Dutcher improved matured filmmaker The film began shaky acting I thought well maybe get better Unfortunately never did The picture starts introducing two elders Mormon missionaries seems audience get know grow care them Instead story degenerates highly improbable series unfortunate events highlighting blatant disobedience missionaries something undeniably exists rarely level Dutcher portrays becomes almost laughable Dutchers success movie successful alienation target audience By unrealistically inaccurately portraying lives Mormon missionaries Dutcher accomplishes nothing angering predominantly Mormon audience The film way reflects reality Missions nothing like Dutcher shows having served Mormon mission I attest fact gang life California certainly contains much explicit language occasional mild vulgarity The conclusion Im assuming supposed touch audience inspire believe forgiveness available all un

In [74]:
imdb_50k['review'][4]

'Petter Mattei\'s "Love in the Time of Money" is a visually stunning film to watch. Mr. Mattei offers us a vivid portrait about human relations. This is a movie that seems to be telling us what money, power and success do to people in the different situations we encounter. <br /><br />This being a variation on the Arthur Schnitzler\'s play about the same theme, the director transfers the action to the present time New York where all these different characters meet and connect. Each one is connected in one way, or another to the next person, but no one seems to know the previous point of contact. Stylishly, the film has a sophisticated luxurious look. We are taken to see how these people live and the world they live in their own habitat.<br /><br />The only thing one gets out of all these souls in the picture is the different stages of loneliness each one inhabits. A big city is not exactly the best place in which human relations find sincere fulfillment, as one discerns is the case wit